In [235]:
import TAGS, progressbar, pickle, re, time, json

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from pprint import pprint
from pathlib import Path
from random import randint

from IPython.display import clear_output

import unicodedata as ud
import pandas as pd

latin_letters = {}

def is_latin(uchr):
    try:
        return latin_letters[uchr]
    except KeyError:
        return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))


def only_roman_chars(unistr):
    return all(is_latin(uchr)
               for uchr in unistr
               if uchr.isalpha())  # isalpha suggested by John Machin


def log(message, level=0):
    if level > 0:
        print(message)

In [ ]:
tags = TAGS.document(path="../../datasets/tags-tsv/boylesque/TAGS - boylesque - Archive.tsv", suppress_warnings=True)
ids = tags.ids
tweets = TAGS.TweetSet(ids, suppress_warnings=True)

100% (32697 of 32697) |##################| Elapsed Time: 0:00:35 Time:  0:00:35


In [176]:
print(len(all_locations))
with open("locations.txt", "w+") as f:
    f.write("\n".join(all_locations))

2167


In [ ]:
# pandas.DataFrame.from_dict(all_data).to_csv('boylesque-dataset-Oct-8-10.00.csv')

In [177]:
with open("locations.txt", "r") as f:
    all_locations = f.read().split("\n")

In [652]:
location_cache_directory = Path('./locations/')

def get_location_data(location, geolocator="Nominatim"):
    if geolocator == "Nominatim":
        log("Geolocator chosen: Nominatim.")
        sleep_val = randint(2,5)
        log(f"Sleeping for {sleep_val} seconds...")
        time.sleep(sleep_val)
        _geolocator = Nominatim(user_agent="tags-research-app")
    else:
        raise NotImplementedError("You can currently only use Nomatim as geolocator in this script.")
    
    try:
        geocoded_location = _geolocator.geocode(location)
    
        if geocoded_location is not None:
            data = {
                'lat': geocoded_location.raw.get('lat', None),
                'lng': geocoded_location.raw.get('lon', None),
                'geolocator': geolocator,
                'type': geocoded_location.raw.get('type', None),
                'class': geocoded_location.raw.get('class', None),
                'display_name': geocoded_location.raw.get('display_name', None),
                'boundingbox': geocoded_location.raw.get('boundingbox', None)
            }
        else:
            data = {
                'lat': None, 'lng': None, 'geolocator': None, 'type': None, 'class': None, 'display_name': None, 'boundingbox': None
            }
    except:
        data = {'error': 'timeout'}
    
    return(data)
    
def read_cache(cache_file):
    with open(cache_file, 'r') as f:
        data = json.load(f)
    return(data)

def save_cache(data, cache_file):
    if data is None or not isinstance(data, dict): raise RuntimeError("Data must not be empty and must be in dictionary format.")
    try:
        with open(cache_file, 'w+') as f:
            json.dump(data, f)
        return(True)
    except:
        return(False)
    
def clean_location(location):
    if re.findall("\w,\w", location):
        location = re.sub(r"(\w+),(\w+)", r"\1, \2", location)
        location = re.sub(r"(\w+),(\w+)", r"\1, \2", location)
        location = re.sub(r"(\w+),(\w+)", r"\1, \2", location)
        location = re.sub(r"(World)|(Main part→)", r"", location, re.I)
    location = TAGS.clean_text(location, no_digits=False) # cleaning up the location
    location = location.lower()
    return(location)

# Set up location_cache_directory
if not location_cache_directory.exists(): location_cache_directory.mkdir()

# Set up standard variables
errors = 0
max_errors = 20

all_locations = []
all_data = []
bar = progressbar.ProgressBar(max_value=len(tweets.tweets)).start()
for i, tweet in enumerate(tweets.tweets):
    inferred = False
    bar.update(i)
    full_text = tweet._json['full_text'].replace("amp", "")
    keywords = TAGS.clean_text(full_text)
    if 'boylesque' in keywords and not keywords.startswith("rt"): # only include tweets mentioning boylesque + not retweets
        geocoder_timeout = False
        
        # Geolocation
        if tweet.geo:
            lat = tweet.geo['coordinates'][0]
            lng = tweet.geo['coordinates'][1]
            inferred = False
            geocoder = None
        elif tweet.user.location is not None and only_roman_chars(tweet.user.location):
            location = clean_location(tweet.user.location)
            if len(location) == 0: next
            cache_file = location_cache_directory / Path(location + ".json")
            if cache_file.is_file():
                geo_data = read_cache(cache_file)
                if geo_data.get('lat') == None and geo_data.get('lng') == None and geo_data.get('boundingbox') == None: next
                lat = geo_data.get('lat', None)
                lng = geo_data.get('lng', None)
                inferred = True
            else:
                geo_data = get_location_data(location)
                inferred = True
                if not 'error' in geo_data:
                    if not save_cache(geo_data, cache_file): raise RuntimeError("File could not be saved.")
                    log(f"Saved: {geo_data} in {cache_file}", 1)
                else:
                    errors += 1
                    log(f"Error: {location} has encountered an error ({geo_data['error']}). (error {errors}/{max_errors})", 1)
                    if errors > max_errors: break
        elif tweet.user.location is not None and not only_roman_chars(tweet.user.location):
            pass # or:
                    # print(f"Location contained non-roman characters: {tweet.user.location}")
        
        data = {
            'keywords': keywords,
            'bounding_box': geo_data.get('boundingbox', None),
            'lat': lat,
            'lng': lng,
            'inferred': inferred,
            'date': time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet.created_at,'%a %b %d %H:%M:%S +0000 %Y')),
            'lang': tweet.lang
        }
        all_data.append(data)
bar.finish()

 24% (8162 of 32697) |####               | Elapsed Time: 0:00:10 ETA:   0:00:35

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict(all_data)
df.to_csv('boylesque-geodata-october-2019.csv')

In [371]:
from shapely.geometry import Point
from shapely.geometry import box

point = Point(0.5, 0.5)
b = box(-75.7854388,-75.6325211,4.3951997,4.5895768)
print(b.contains(point))

True


In [ ]:
(-75.7854388, -75.6325211),
(4.3951997, 4.5895768),

In [546]:
# Bounding box is: "south Latitude, north Latitude, west Longitude, east Longitude"
# according to https://wiki.openstreetmap.org/wiki/Bounding_Box
for file in Path('./locations/').glob('colchester.json'):
    with file.open() as f:
        data = json.load(f)
        if data['boundingbox'] is not None:
            min_lat = data['boundingbox'][0]
            max_lat = data['boundingbox'][1]
            min_lng = data['boundingbox'][2]
            max_lng = data['boundingbox'][3]
            print(file, data['display_name'])
            print(f"(min_lng,max_lng,min_lat,max_lat)")
            print(f"({min_lng},{max_lng},{min_lat},{max_lat})")
            print(data['lat'], data['lng'])
            print("")

locations/colchester.json Colchester, Essex, East of England, England, United Kingdom
(min_lng,max_lng,min_lat,max_lat)
(0.6993788,1.0268034,51.7657055,51.9771532)
51.8896903 0.8994651



In [655]:
def get_boundingbox(location = None):
    if not location: raise RuntimeError(f"Location must be set to not None.")
    location = clean_location(tweet.user.location)
    if len(location):
        cache_file = location_cache_directory / Path(location + ".json")
        if cache_file.is_file():
            geo_data = read_cache(cache_file)
            return_data = geo_data.get('boundingbox', None)
            if isinstance(return_data, list): 
                print("return_data is a list!")
                return_data = [float(x) for x in return_data] # returned as list of floats: [NS/latitude min, NS/latitude max; EW/longitude min; EW/longitude max]
            return(return_data)
        else:
            log(f"Cannot read cache file / file does not exist: {cache_file}")
            return(None)

def get_user_location(tweet):
    if tweet.user.location is not None:
        location = clean_location(tweet.user.location)
    else:
        return(False)
    
    if not location or len(location) == 0 or not only_roman_chars(location): # for now, this is the implementation
        return(False)
    else:
        cache_file = location_cache_directory / Path(location + ".json")
        if cache_file.is_file():
            geo_data = read_cache(cache_file)
            lat = geo_data.get('lat', None)
            lng = geo_data.get('lng', None)
            if lat: lat = float(lat)
            if lng: lat = float(lng)
            return([lat, lng])
        else:
            log(f"Cannot read cache file / file does not exist: {cache_file}", 1)
            return(False)

def get_native_tweet_location(tweet):
    if tweet.geo is not None and tweet.geo.get('coordinates', None):
        coordinates = tweet.geo.get('coordinates', None)
        if coordinates: coordinates = [float(x) for x in coordinates]
        return(coordinates)
    else:
        return(False)

def return_keywords_in_bounding_box(tweets, location = None, boundingbox = [], include_inferred = True, filter = True):
    all_keywords, total_processed_tweets = {}, 0
    if not location and not boundingbox: raise RuntimeError("Location or boundingbox must be provided.")
    if location and boundingbox: raise RuntimeError("Location or boundingbox must be provided, not both.")

    # We have been provided with location, let's extract boundingbox
    if location: boundingbox = get_boundingbox(location = location)
        
    # We have been provided with boundingbox, let's make sure datatypes are correct
    if boundingbox and isinstance(boundingbox, list): boundingbox = [float(x) for x in boundingbox]
    
    if filter:
        total_unfiltered_tweets = len(tweets)
        log(f"Filtering {total_unfiltered_tweets} tweets.", 1)
        tweets = [x for x in tweets 
                      if not TAGS.clean_text(x.full_text).startswith("rt") 
                      and "boylesque" in TAGS.clean_text(x.full_text)
                 ]
        total_num_tweets = len(tweets)
        log(f"=> {total_num_tweets} filtered tweets.", 1)
    
    bar = progressbar.ProgressBar(maxval=total_num_tweets).start()
    for i, tweet in enumerate(tweets):
        clear_output()
        bar.update(i)
        if get_native_tweet_location(tweet):
            lat, lng = get_native_tweet_location(tweet)[0], get_native_tweet_location(tweet)[1]
        elif include_inferred and get_user_location(tweet):
            lat, lng = get_user_location(tweet)[0], get_user_location(tweet)[1]
        else:
            lat, lng = None, None

        # check if lat + lng + boundingbox exists
        if lat and lng and boundingbox:
            # TODO: test for type not float....!
            if (boundingbox[1] > lat > boundingbox[0]) and (boundingbox[3] > lng > boundingbox[2]):
                full_text = tweet.full_text.replace("amp", "")
                keywords = TAGS.clean_text(full_text)

                for keyword in keywords.split(" "):
                    if keyword not in all_keywords: all_keywords[keyword] = 0
                    all_keywords[keyword] += 1
                total_processed_tweets += 1
    bar.finish()

    return({'total_unfiltered_tweets': total_unfiltered_tweets, 'total_num_tweets': total_num_tweets, 'total_processed_tweets': total_processed_tweets, 'all_keywords': all_keywords})

In [656]:
# return_keywords_in_bounding_box(tweets.tweets, location = "colchester") # retrieve all the keywords from colchester..
# return_keywords_in_bounding_box(tweets.tweets, location='Austin, TX') # = retrieves all keywords from Austin, TX
# you can also use include_inferred parameter to exclude those tweets where location is inferred through the user's fill-in location [although I am not sure that works just yet]

# [min_lat, max_lat, min_lng, max_lng]
# A couple of boundingboxes:
#  - UK: ['51.765706', '59.645540', '-11.601563', '1.026803']
#  - US: ['24.76' ,'49.21', '-128.232422', '-66.88']
#       east coast: ['25', '49.21', '-81', '-66.5']
#       west coast: ['31', '49.21', '-128', '-116']


# Example: Get all keywords from tweets from UK: return_keywords_in_bounding_box(tweets.tweets, boundingbox=['51.765706', '59.645540', '-11.601563', '1.026803'])
# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['51.765706' ,'59.645540', '-128.232422', '1.026803'])

# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['24.76' ,'49.21', '-128.232422', '-66.88']) # = retrieves all keywords from the U.S... not very precise
# return_keywords_in_bounding_box(tweets.tweets, boundingbox=['31', '49.21', '-128', '-116']) # = retrieves keywords from the west coast of U.S. 
return_keywords_in_bounding_box(tweets.tweets, boundingbox=['25', '49.21', '-81', '-66.5']) # = retrieves keywords from the east coast of U.S. 


TypeError: '>' not supported between instances of 'float' and 'str'